In [ ]:
!pip install torch torchvision timm

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import timm
import time

In [ ]:
# Configurações iniciais
batch_size = 32
learning_rate = 0.001
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Transformação do dataset
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Converte para 3 canais
    transforms.Resize((128, 128)),  # Redimensiona para 224x224
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [ ]:
# Carregamento do dataset Fashion-MNIST
train_dataset = datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.FashionMNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

100%|██████████| 26.4M/26.4M [00:02<00:00, 10.8MB/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 174kB/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:01<00:00, 3.19MB/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 6.06MB/s]


Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [ ]:
# Função de treinamento
def train(model, loader, criterion, optimizer):
    model.train()
    total_loss = 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(loader)

In [ ]:
# Função de avaliação
def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    return accuracy

In [ ]:
# Função principal para treinar e avaliar os modelos
def train_and_evaluate(model_name, pre_training):
    # Carrega o modelo do timm e ajusta para 10 classes (Fashion-MNIST)
    model = timm.create_model(model_name, pretrained=pre_training, num_classes=10).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    print(f"\nTreinando o modelo: {model_name}")
    start_time = time.time()

    for epoch in range(num_epochs):
        train_loss = train(model, train_loader, criterion, optimizer)
        test_accuracy = evaluate(model, test_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

    end_time = time.time()
    print(f"Tempo total de treinamento para {model_name}: {(end_time - start_time) / 60:.2f} minutos\n")

In [ ]:
pre_trained_model = 'tf_efficientnetv2_s.in21k'  # EfficientNetV2
raw_model = 'efficientnetv2_s'

train_and_evaluate(pre_trained_model, pre_training=True)

print(f"FINALIZADO {pretrained_model}\n")

train_and_evaluate(raw_model, pre_training=False)

print(f"FINALIZADO {raw_model}\n")


Treinando o modelo: efficientnetv2_s
Epoch [1/10], Loss: 0.5969, Test Accuracy: 0.8652
Epoch [2/10], Loss: 0.3742, Test Accuracy: 0.8811
Epoch [3/10], Loss: 0.2945, Test Accuracy: 0.8953
Epoch [4/10], Loss: 0.2552, Test Accuracy: 0.9050
Epoch [5/10], Loss: 0.2351, Test Accuracy: 0.9084
Epoch [6/10], Loss: 0.2098, Test Accuracy: 0.9204
Epoch [7/10], Loss: 0.1871, Test Accuracy: 0.9150
Epoch [8/10], Loss: 0.1688, Test Accuracy: 0.9201
Epoch [9/10], Loss: 0.1476, Test Accuracy: 0.9249
Epoch [10/10], Loss: 0.1269, Test Accuracy: 0.9295
Tempo total de treinamento para efficientnetv2_s: 52.03 minutos

FINALIZADO efficientnetv2_s

